In [1]:
!pip install streamlit
!pip install streamlit_drawable_canvas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 1.8 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 5.7 MB/s eta 0:00:00


In [36]:
%%writefile app.py
import streamlit as st
import cv2
import torch
import numpy as np
import streamlit as st
from streamlit_drawable_canvas import st_canvas
#from models import *
import plotly.graph_objects as go
import torch
import torch.nn as nn

#model
# Define a custom neural network class that inherits from nn.Module
class MyModel(nn.Module):
    def __init__(self):
        # Call the constructor of the parent class (nn.Module)
        super(MyModel, self).__init__()

        # Define the convolutional block of the neural network
        self.cnn_block = nn.Sequential(
            # First convolutional layer with 1 input channel and 8 output channels
            nn.Conv2d(in_channels=1, out_channels=8, kernel_size=(3,3), padding=1),
            # Batch normalization for the 8 output channels of the first convolutional layer
            nn.BatchNorm2d(8),
            # ReLU activation function after the first convolutional layer
            nn.ReLU(),
            # Second convolutional layer with 8 input channels and 16 output channels
            nn.Conv2d(in_channels=8, out_channels=16, kernel_size=(3,3), padding=1),
            # Batch normalization for the 16 output channels of the second convolutional layer
            nn.BatchNorm2d(16),
            # ReLU activation function after the second convolutional layer
            nn.ReLU(),
            # Max pooling layer that reduces the spatial dimensions by a factor of 2
            nn.MaxPool2d(kernel_size=(2,2)),
            # Third convolutional layer with 16 input channels and 32 output channels
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=(3,3), padding=1),
            # Batch normalization for the 32 output channels of the third convolutional layer
            nn.BatchNorm2d(32),
            # ReLU activation function after the third convolutional layer
            nn.ReLU(),
            # Another max pooling layer that further reduces the spatial dimensions by a factor of 2
            nn.MaxPool2d(kernel_size=(2,2))
        )

        # Define the linear (fully connected) block of the neural network
        self.linear_block = nn.Sequential(
            # Flatten the output of the convolutional block to make it suitable for the linear layers
            nn.Flatten(),
            # First linear layer that takes the flattened input and outputs 512 features
            nn.Linear(32*7*7, 512),
            # ReLU activation function after the first linear layer
            nn.ReLU(),
            # Dropout layer with a dropout probability of 0.5 to reduce overfitting
            nn.Dropout(p=0.5),
            # Second linear layer that takes the 512 input features and outputs 256 features
            nn.Linear(512, 256),
            # ReLU activation function after the second linear layer
            nn.ReLU(),
            # Another dropout layer with a dropout probability of 0.5 to further reduce overfitting
            nn.Dropout(p=0.5),
            # Final linear layer that takes the 256 input features and outputs 10 features,
            # which correspond to the class scores for a 10-class classification problem
            nn.Linear(256, 10)
        )

    def forward(self, images):
        # Define the forward pass through the network
        # First, pass the input images through the convolutional block
        x = self.cnn_block(images)
        # Then, pass the output of the convolutional block through the linear block
        logits = self.linear_block(x)
        # Return the final class logits
        return logits


model= MyModel()


# Display the title of the app
st.title('Digit Recognizer')

# Create two columns with equal width for layout
col1, col2 = st.columns([1, 1])

# Column 1: User input section
with col1:
    # Add some space before the checkbox for better layout
    for i in range(5):
        st.write(' ')

    # Checkbox to toggle between drawing and transforming mode
    mode = st.checkbox("Draw or Transform", True)

    # Streamlit Canvas for user input drawing
    # Users can draw digits on this canvas
    canvas_result = st_canvas(
        fill_color='#000000',  # Canvas fill color
        stroke_width=20,  # Width of the drawing pen
        stroke_color='#FFFFFF',  # Color of the pen
        background_color='#000000',  # Background color of the canvas
        width=256,  # Width of the canvas
        height=256,  # Height of the canvas
        drawing_mode="freedraw" if mode else "transform",  # Drawing mode
        key='canvas'
    )

    # Button for predicting the digit drawn on the canvas
    pred_button = st.button('Predict')

# Process the image from the canvas if it exists
if canvas_result.image_data is not None:
    # Resize the canvas image to 28x28 pixels for the model input
    img = cv2.resize(canvas_result.image_data.astype('uint8'), (28, 28))

    # Convert the image to grayscale since the model expects grayscale input
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # Resulting shape (28, 28)

    # Normalize the image by dividing by 255 and add a channel dimension
    img = img[:, :, np.newaxis] / 255.0

# Column 2: Prediction and result visualization section
with col2:
    if pred_button:
        # Disable gradient calculations for prediction (inference mode)
        with torch.no_grad():
            # Convert the processed image to a PyTorch tensor and adjust dimensions
            # The expected dimension is (batch_size, channels, height, width)
            img = torch.Tensor(img).permute(2, 0, 1)

            # Get model logits for the input image
            logits = model(img.unsqueeze(0))  # Add a batch dimension

            # Apply softmax to convert logits to probabilities
            sm = torch.nn.Softmax(dim=1)
            probs = sm(logits)[0]  # Get probabilities for the first image in the batch

            # Create a bar plot of the digit probabilities using Plotly
            fig = go.Figure(
                data=[
                    go.Bar(
                        x=np.arange(0, 10),  # X-axis labels (0-9 digits)
                        y=(probs * 100).numpy()  # Y-axis values as probabilities
                    )
                ]
            )

            # Update figure layout for better visualization
            fig.update_layout(
                width=500,  # Width of the plot
                height=500  # Height of the plot
            )

            # Display the Plotly bar plot in the Streamlit app
            st.plotly_chart(fig)


Overwriting app.py


In [37]:
!npm install localtunnel

npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ localtunnel@2.0.2
updated 1 package and audited 36 packages in 0.854s

3 packages are looking for funding
  run `npm fund` for details

found 2 moderate severity vulnerabilities
  run `npm audit fix` to fix them, or `npm audit` for details


In [38]:
!streamlit run app.py &>/content/logs.txt &


In [39]:
import urllib
print("Password/Enpoint IP for localtunnel is:",urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))

Password/Enpoint IP for localtunnel is: 104.197.174.137


In [40]:
!npx localtunnel --port 8501

npx: installed 22 in 2.411s
your url is: https://breezy-rabbits-sort.loca.lt
^C
